# A Short MACE Tutorial
## Ilyes Batatia 

Hot link to Google Colab version: https://colab.research.google.com/drive/1D6EtMUjQPey_GkuxUAbPgld6_9ibIa-V?authuser=0&pli=1#scrollTo=X2XNYxlFHEKR

## Introduction

This is a short tutorial for MACE, a highly accurate and efficient ML interatomic potential.
Please read the associated [paper](https://arxiv.org/pdf/2206.07697.pdf).
The reference implementation is available [here](https://github.com/ACEsuit/mace).

## Installation

In [ ]:
# Install dependencies
!pip install e3nn==0.4.4 opt_einsum ase torch_ema prettytable

# Clone MACE
!git clone --depth 1 https://github.com/ACEsuit/mace.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 387.7/387.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [ ]:
!pip install mace/

Processing ./mace
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 3.1 MB/s eta 0:00:00
  Created wheel for mace-torch: filename=mace_torch-0.3.4-py3-none-any.whl size=87188 sha256=09450eae8a778c0bdccf4f218e2f4af17123a0bc35d10bcb3de28b0fa4fa7ef9
  Stored in directory: /tmp/pip-ephem-wheel-cache-up82ga0y/wheels/df/5f/32/ef59561725170a81c728fd01c75e56a9ee83bad6da485fc6a5
Successfully built mace-torch


**Note:** Make sure to enable GPU: Runtime --> Change runtime type to GPU

## Loading Data
The data files used to train the MACE model have to be in `extxyz` format.
In this tutorial, we use the 3BPA dataset consisting of 500 configurations sampled a 300K with DFT.
The energies are in eV and forces in eV/A.

In [ ]:
!git clone https://github.com/davkovacs/BOTNet-datasets.git

Cloning into 'BOTNet-datasets'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 57 (delta 13), reused 37 (delta 7), pack-reused 0
Receiving objects: 100% (57/57), 28.73 MiB | 20.03 MiB/s, done.
Resolving deltas: 100% (13/13), done.


In [ ]:
!ls BOTNet-datasets/dataset_3BPA

iso_atoms.xyz  test_1200K.xyz  test_600K.xyz  train_300K.xyz
README.md      test_300K.xyz   test_dih.xyz   train_mixedT.xyz


## Training

To train a MACE model you can specify the training file with the `--train_file` flag. The validation set can either be specified as a separate file using the `--valid_file` keyword, or it can be specified as a fraction of the training set using the `--valid_fraction` keyword. It is also possible to provide a test set that only gets evaluated at the end of the training using the `--test_file` keyword. If you want to compute the RMSE for different parts of the training set separately, specify the `config_type` keyword in the `info` dict of the configurations.

When parsing the data files the energies are read using the keyword `energy` and the forces using the keyword `forces`. To change that, specify the `--energy_key` and `--forces_key`.

For illustration, we create a very small model with 16 invariant messages specified by `hidden_irreps='16x0e'`.

In [ ]:
!python3 ./mace/scripts/run_train.py \
  --name="MACE_model" \
  --train_file="BOTNet-datasets/dataset_3BPA/train_300K.xyz" \
  --valid_fraction=0.05 \
  --test_file="BOTNet-datasets/dataset_3BPA/test_300K.xyz" \
  --E0s='{1:-13.663181292231226, 6:-1029.2809654211628, 7:-1484.1187695035828, 8:-2042.0330099956639}' \
  --model="ScaleShiftMACE" \
  --hidden_irreps='32x0e' \
  --r_max=4.0 \
  --batch_size=20 \
  --max_num_epochs=100 \
  --ema \
  --ema_decay=0.99 \
  --amsgrad \
  --default_dtype="float32" \
  --device=cpu \
  --seed=123 \
  --swa

2024-04-02 10:00:25.893 INFO: MACE version: 0.3.4
2024-04-02 10:00:25.895 INFO: Configuration: Namespace(name='MACE_model', seed=123, log_dir='logs', model_dir='.', checkpoints_dir='checkpoints', results_dir='results', downloads_dir='downloads', device='cpu', default_dtype='float32', log_level='INFO', error_table='PerAtomRMSE', model='ScaleShiftMACE', r_max=4.0, radial_type='bessel', num_radial_basis=8, num_cutoff_basis=5, interaction='RealAgnosticResidualInteractionBlock', interaction_first='RealAgnosticResidualInteractionBlock', max_ell=3, correlation=3, num_interactions=2, MLP_irreps='16x0e', radial_MLP='[64, 64, 64]', hidden_irreps='32x0e', num_channels=None, max_L=None, gate='silu', scaling='rms_forces_scaling', avg_num_neighbors=1, compute_avg_num_neighbors=True, compute_stress=False, compute_forces=True, train_file='BOTNet-datasets/dataset_3BPA/train_300K.xyz', valid_file=None, valid_fraction=0.05, test_file='BOTNet-datasets/dataset_3BPA/test_300K.xyz', E0s='{1:-13.6631812922312

It is possible to use `--model=MACE`, in order to have the correct limit for isolated atoms. This is recommanded for task studying bond breaking events.

## Run

The trained model is realidy usable to run some ASE MD for illustration. The Colab hardware are not very performant so we put a small number of timesteps for illustration.

In [ ]:
from ase import units
from ase.md.langevin import Langevin
from ase.io import read, write
import numpy as np
import time

from mace.calculators import MACECalculator

calculator = MACECalculator(model_paths='/content/checkpoints/MACE_model_run-123.model', device='cpu')
init_conf = read('BOTNet-datasets/dataset_3BPA/test_300K.xyz', '0')
init_conf.set_calculator(calculator)

dyn = Langevin(init_conf, 0.5*units.fs, temperature_K=310, friction=5e-3)
def write_frame():
        dyn.atoms.write('md_3bpa.xyz', append=True)
dyn.attach(write_frame, interval=50)
dyn.run(100)
print("MD finished!")